In [6]:
import pandas as pd
from twarc import Twarc
import datetime

In [7]:
days = list()
for x in range(101):
    days.append("day_{}_time".format(x))
    days.append("day_{}_retweet_count".format(x))

columns = ["screen_name", "tweet_id","created_at","embeded_url","expanded_url","author","title","content"] + days
columns

['screen_name',
 'tweet_id',
 'created_at',
 'embeded_url',
 'expanded_url',
 'author',
 'title',
 'content',
 'day_0_time',
 'day_0_retweet_count',
 'day_1_time',
 'day_1_retweet_count',
 'day_2_time',
 'day_2_retweet_count',
 'day_3_time',
 'day_3_retweet_count',
 'day_4_time',
 'day_4_retweet_count',
 'day_5_time',
 'day_5_retweet_count',
 'day_6_time',
 'day_6_retweet_count',
 'day_7_time',
 'day_7_retweet_count',
 'day_8_time',
 'day_8_retweet_count',
 'day_9_time',
 'day_9_retweet_count',
 'day_10_time',
 'day_10_retweet_count',
 'day_11_time',
 'day_11_retweet_count',
 'day_12_time',
 'day_12_retweet_count',
 'day_13_time',
 'day_13_retweet_count',
 'day_14_time',
 'day_14_retweet_count',
 'day_15_time',
 'day_15_retweet_count',
 'day_16_time',
 'day_16_retweet_count',
 'day_17_time',
 'day_17_retweet_count',
 'day_18_time',
 'day_18_retweet_count',
 'day_19_time',
 'day_19_retweet_count',
 'day_20_time',
 'day_20_retweet_count',
 'day_21_time',
 'day_21_retweet_count',
 'day_22

## Should we add favorite_count as well?

In [8]:
df = pd.DataFrame(columns=columns)
df.head()

def export_dataset(df):
    name = "../data/dataset_{}.csv".format(datetime.datetime.now().strftime("%B_%d_%y_%H_%M_%S"))
    df.to_csv(name)

# export_dataset(df)

In [9]:
consumer_key='58wlGpzEOwum6nghvA8tj6nz6'
consumer_secret='b04KqyzRyquIiYGwqv9B12hRqjEj9tvOp53DtoFZJtetaZ7cZE'
access_token='1243275961824612358-65qxIlXTEhjw0jlbBaK4I0olH9nRP9'
access_token_secret='wYdkYaNCakb7sX5PnHjF1u0oJc7Y73iijqbJ4VeB4yyEd'
 
t = Twarc(consumer_key, consumer_secret, access_token, access_token_secret)

a tweet is a valid tweet if
1. it has user\[id\} belonging to the list of listed ids and has embeded url
2. **if has a retweet that satisfies criteria 1 and the source tweet is published today**

In [18]:
# tweets = set()
actually_added = 0
parsed = 0
max_limit = 20
follow_list = ['807095']
follow_str = ",".join(follow_list)
print("Following users ",follow_str)
for tweet in t.filter(follow=follow_list):
    parsed += 1
    # tweet for a user we are following
    if tweet['user']['id'] in follow_list \
#     TODO: Code to remove retweets
        and 'entities' in tweet \
        and tweet['entities']['urls'] != [] \
        and 'url' in tweet['entities']['urls'][0]\
        and (df['tweet_id'] != tweet['id']).all():
        print("From source", tweet['id'])
        actually_added += 1
        
        df = df.append({'screen_name':tweet['user']['screen_name'], 
              'tweet_id':tweet['id'],
              'created_at' : tweet['created_at'],
              'embeded_url' : tweet['entities']['urls'][0]['url'],
              'day_0_retweet_count': tweet['retweet_count'],
              'day_0_time':datetime.datetime.now()}, ignore_index=True)

    if 'retweeted_status' in tweet \
        and 'extended_tweet' in tweet['retweeted_status'] \
        and 'entities' in tweet["retweeted_status"]["extended_tweet"] \
        and 'urls' in tweet["retweeted_status"]["extended_tweet"]["entities"]\
        and (df['tweet_id'] != tweet['retweeted_status']['id']).all():
        
        created_time = datetime.datetime.strptime(tweet["retweeted_status"]["created_at"],"%a %b %d %H:%M:%S %z %Y")
        current = datetime.datetime.now(datetime.timezone.utc)
        diff =  current - created_time
        print(diff)
        if diff.days == 0:
            print("From retweet", tweet['id'],"adding tweet : ",tweet["retweeted_status"]["id"])
            df = df.append({'screen_name':tweet["retweeted_status"]["user"]["screen_name"], 
                  'tweet_id':tweet["retweeted_status"]["id"],
                  'created_at' : tweet["retweeted_status"]["created_at"],
                  'embeded_url' : tweet["retweeted_status"]["extended_tweet"]["entities"]["urls"][0]["url"],
                  'day_0_retweet_count': tweet['retweeted_status']['retweet_count'],
                  'day_0_time': current}, ignore_index=True)
            actually_added += 1
#             print(tweet['created_at'])
#             print(tweet['id'])
#             print(tweet['user']['id'])
#             print(tweet["retweeted_status"]["id"])
#             print(tweet["retweeted_status"]["created_at"])
#             print(tweet["retweeted_status"]["user"]["screen_name"])
#             print(tweet["retweeted_status"]["extended_tweet"]["entities"]["urls"][0]["url"])
    if actually_added == max_limit:
        break
print("Parsed :",parsed,"\n Added : ",actually_added)

SyntaxError: invalid syntax (<ipython-input-18-f28132a75b56>, line 12)

In [17]:
df.head(max_limit)

,screen_name,tweet_id,created_at,embeded_url,expanded_url,author,title,content,day_0_time,day_0_retweet_count,...,day_96_time,day_96_retweet_count,day_97_time,day_97_retweet_count,day_98_time,day_98_retweet_count,day_99_time,day_99_retweet_count,day_100_time,day_100_retweet_count
0,nytimes,1270044909886672896,Mon Jun 08 17:27:57 +0000 2020,https://t.co/2ZlOW9YXDZ,NaN,NaN,NaN,NaN,2020-06-08 19:36:17.904694,1062,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,nytimes,1270128497743343628,Mon Jun 08 23:00:06 +0000 2020,https://t.co/iX9bNpATMW,NaN,NaN,NaN,NaN,2020-06-08 23:37:16.124164+00:00,176,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,nytimes,1270038622859919364,Mon Jun 08 17:02:58 +0000 2020,https://t.co/ZDJ091U7fj,NaN,NaN,NaN,NaN,2020-06-08 23:37:22.871892+00:00,1004,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,nytimes,1270093170479677441,Mon Jun 08 20:39:43 +0000 2020,https://t.co/p1PBh8KxNG,NaN,NaN,NaN,NaN,2020-06-08 23:37:25.374499+00:00,928,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,nytimes,1270017767186534400,Mon Jun 08 15:40:06 +0000 2020,https://t.co/yO85O14Gn8,NaN,NaN,NaN,NaN,2020-06-08 23:37:28.109188+00:00,843,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,nytimes,1270133513891840002,Mon Jun 08 23:20:02 +0000 2020,https://t.co/XCvQDk49kb,NaN,NaN,NaN,NaN,2020-06-08 23:37:28.876738+00:00,160,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,nytimes,1270050491603521536,Mon Jun 08 17:50:08 +0000 2020,https://t.co/OxucrDP2gB,NaN,NaN,NaN,NaN,2020-06-08 23:37:48.988105+00:00,1096,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,nytimes,1270058022988021769,Mon Jun 08 18:20:03 +0000 2020,https://t.co/h7KQdlBq5v,NaN,NaN,NaN,NaN,2020-06-08 23:37:51.853078+00:00,160,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,nytimes,1270118431304617986,Mon Jun 08 22:20:06 +0000 2020,https://t.co/BGzHX3gp8c,NaN,NaN,NaN,NaN,2020-06-08 23:37:52.727926+00:00,196,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,nytimes,1269972462147682304,Mon Jun 08 12:40:04 +0000 2020,https://t.co/olSEnLvWPa,NaN,NaN,NaN,NaN,2020-06-08 23:38:01.077370+00:00,1573,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
for tweet in t.hydrate([1270044909886672896]):
    tweet['retweet_count']
#     print(tweet)
    print(tweet['user']['screen_name'])
    print(tweet['id'])
    print(tweet['created_at'])
#     print(tweet['retweeted_status']['id'])
#     print(tweet['entities']['urls'][0]['url'])
#     print(tweet['retweeted_status'])
    print(tweet['favorite_count'])
#     print("retweeted_status count", tweet['retweeted_status']['retweet_count'])
    print(datetime.datetime.now())
    df = df.append({'screen_name':tweet['user']['screen_name'], 
              'tweet_id':tweet['id'],
              'created_at' : tweet['created_at'],
              'embeded_url' : tweet['entities']['urls'][0]['url'],
              'day_0_retweet_count': tweet['retweet_count'],
              'day_0_time':datetime.datetime.now()}, ignore_index=True)
created_time = datetime.datetime.strptime(tweet['created_at'],"%a %b %d %H:%M:%S %z %Y")
diff =  datetime.datetime.now(datetime.timezone.utc) - created_time
abs(diff.days), datetime.datetime.now(datetime.timezone.utc), created_time

nytimes
1270044909886672896
Mon Jun 08 17:27:57 +0000 2020
2503
2020-06-08 19:36:17.904694


(0,
 datetime.datetime(2020, 6, 8, 23, 36, 17, 929626, tzinfo=datetime.timezone.utc),
 datetime.datetime(2020, 6, 8, 17, 27, 57, tzinfo=datetime.timezone.utc))

In [50]:
# export_dataset(df)

In [15]:
if (df['tweet_id'] == tweet['id']).any():
    print("yes")

In [55]:
df.head()

,screen_name,tweet_id,created_at,embeded_url,expanded_url,author,title,content,day_0_time,day_0_retweet_count,...,day_96_time,day_96_retweet_count,day_97_time,day_97_retweet_count,day_98_time,day_98_retweet_count,day_99_time,day_99_retweet_count,day_100_time,day_100_retweet_count
0,nytimes,1270083188753338372,Mon Jun 08 20:00:03 +0000 2020,https://t.co/IJ6g1Y0ZhZ,NaN,NaN,NaN,NaN,2020-06-08 18:39:44.159104,60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,nytimes,1270083188753338371,Mon Jun 08 20:00:03 +0000 2020,https://t.co/IJ6g1Y0ZhZ,NaN,NaN,NaN,NaN,2020-06-08 18:40:02.197581,60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
(df['tweet_id'] == tweet['id']).any()

True